### MLP

In [127]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [128]:
df = pd.read_csv("Datasets/new_dataset_added_column_scaled.csv")
# df = pd.read_csv("/Users/asheshlalshrestha/Desktop/UB/Research/CarePathways/Models/Datasets/preprocessed_dataset.csv")

In [129]:
df = df.drop(["Unnamed: 0","id"],axis=1)
# df = df[df['hospital_length_of_stay']<20]

In [130]:
y = df.hospital_length_of_stay
X = df.drop('hospital_length_of_stay',axis=1)

In [131]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [133]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import keras_tuner as kt

# Function to create the model with tunable parameters
def build_model(hp):
    model = Sequential()
    
    # Tune the number of units in the first Dense layer
    model.add(Dense(units=hp.Int('units1', min_value=32, max_value=320, step=16), 
                    activation=hp.Choice('activation1', ['relu', 'tanh', 'sigmoid']), 
                    input_shape=(X_train.shape[1],)))
    
    # Tune the number of units in the second Dense layer
    model.add(Dense(units=hp.Int('units2', min_value=16, max_value=320, step=16), 
                    activation=hp.Choice('activation2', ['relu', 'tanh', 'sigmoid'])))
    
    
    model.add(Dense(1))  # Output layer

    # Compile the model
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    
    return model

# Set up the Keras Tuner with a new directory
tuner = kt.Hyperband(
    build_model,
    objective='val_mae',
    max_epochs=100,
    factor=3,
    directory='new_my_dir',  # Change this to a new directory
    project_name='new_mlp_tuning'  # Change project name to avoid conflicts
)

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_mae', patience=5)

# Run the tuner
tuner.search(X_train, y_train, epochs=100, validation_split=0.2, callbacks=[early_stopping])

# Retrieve the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluate the best model
mae_best, mse_best = best_model.evaluate(X_test, y_test, verbose=0)
print(f"Best Model MSE: {mse_best}")


Trial 253 Complete [00h 00m 02s]
val_mae: 7.236936092376709

Best val_mae So Far: 6.878542900085449
Total elapsed time: 00h 07m 56s
Best Model MSE: 7.104276657104492


In [124]:
# Retrieve the best trial
best_trial = tuner.oracle.get_best_trials(num_trials=1)[0]

# Print the hyperparameters of the best model
print("Best Hyperparameters:")
for hp_name in best_trial.hyperparameters.values.keys():
    print(f"{hp_name}: {best_trial.hyperparameters.get(hp_name)}")

# Optionally, print the summary of the best model architecture
best_model.summary()

# Evaluate the best model
mae_best, mse_best = best_model.evaluate(X_test, y_test, verbose=0)
print(f"Best Model MSE: {mse_best}")

Best Hyperparameters:
units1: 96
units2: 64
units3: 32
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 4
tuner/round: 0
activation1: relu
activation2: relu
activation3: relu


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 96)             │        75,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 83,489 (326.13 KB)

 Trainable params: 83,489 (326.13 KB)

 Non-trainable params: 0 (0.00 B)

Best Model MSE: 8.1699800491333


In [122]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error

# Define the best hyperparameters
best_units = {
    'units1': 96,
    'units2': 64,
    'units3': 32,
}
best_activations = {
    'activation1': 'relu',
    'activation2': 'relu',
    'activation3': 'relu',
}

# Build the model with the best hyperparameters
def build_best_model():
    model = Sequential()
    model.add(Dense(units=best_units['units1'], activation=best_activations['activation1'], input_shape=(X_train.shape[1],)))
    model.add(Dense(units=best_units['units2'], activation=best_activations['activation2']))
    model.add(Dense(units=best_units['units3'], activation=best_activations['activation3']))
    model.add(Dense(1))  # Output layer
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

# Create the best model
best_model = build_best_model()

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_mae', patience=5)

# Train the model on the training data
history = best_model.fit(X_train, y_train, 
                          epochs=100, 
                          validation_split=0.2, 
                          callbacks=[early_stopping],
                          verbose=1)

# Evaluate the best model on the test data
mse_best, mae_best = best_model.evaluate(X_test, y_test, verbose=0)
print(f"Best Model MSE: {mse_best}")

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate MSE manually
mse_calculated = mean_squared_error(y_test, y_pred)
print(f"Calculated MSE: {mse_calculated}")

Epoch 1/100


/Users/asheshlalshrestha/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 298.3049 - mae: 11.8633 - val_loss: 251.1917 - val_mae: 10.9793
Epoch 2/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 228.1345 - mae: 9.8247 - val_loss: 188.5579 - val_mae: 8.2715
Epoch 3/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 172.2934 - mae: 7.6760 - val_loss: 127.1625 - val_mae: 7.3731
Epoch 4/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 123.3468 - mae: 7.8693 - val_loss: 124.6458 - val_mae: 8.3054
Epoch 5/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 161.2224 - mae: 8.9266 - val_loss: 123.3083 - val_mae: 7.4476
Epoch 6/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 96.2622 - mae: 6.4513 - val_loss: 125.1907 - val_mae: 7.2795
Epoch 7/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 99.4248 - mae: 6.6379 - val_loss: 123.4350 - val_mae: 7.6357
Epoch 8/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 86.5954 - mae: 6.4103 - val_loss: 124.2538 - val_mae: 7.7878
Epoch 9/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
Calculated MSE: 138.04015870832842


In [116]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
# Shuffle and slice the dataset.
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

# Now we get a test dataset.
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_dataset = test_dataset.batch(64)

# Since the dataset already takes care of batching,
# we don't pass a `batch_size` argument.
best_model.fit(train_dataset, epochs=3)

# You can also evaluate or predict on a dataset.
print("Evaluate")
result = best_model.evaluate(test_dataset)
dict(zip(best_model.metrics_names, result))

Epoch 1/3
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 57.3116 - mae: 5.3268
Epoch 2/3
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 43.5289 - mae: 4.4592
Epoch 3/3
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 52.3173 - mae: 4.8747
Evaluate
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 138.1231 - mae: 8.4394


{'loss': 144.8095245361328, 'compile_metrics': 8.391454696655273}

In [121]:
best_model.evaluate(X_test, y_test, verbose=0)

[144.8095245361328, 8.391454696655273]